In [ ]:
import pandas as pd
from tqdm import tqdm
from IPython.display import display
import warnings
import pprint
import time
warnings.simplefilter('ignore')

#ファイルのインポート
import evaluation as ev
import control_structure as cs
import race
import table as tb
import init_table as init
import creating_test as test
import os_operation as _os_

RESULTS_FILE_NAME='sample' #.csvつけない

# インポートする dir
path='data_buffer2/'

# エクスポートする dir
results_path='results_buffer2/'


files=_os_.get_files(path)
send_files=files['send_files']
recv_files=files['recv_files']


heading_results=('Task','Buffer','TestCase','SVAR','Def-Use','Plnv')
heading=('Thread','Port','Event','Index')
pre_results=pd.DataFrame({},columns=heading_results)
new_results=pd.DataFrame({},columns=heading_results)

for file in tqdm(range(len(recv_files))):
    print('-------{}回目-------'.format(file+1))
    RECEIVE_SHEET_NAME=path+recv_files[file]
    SEND_SHEET_NAME=path+send_files[file]
    
    Qs=[pd.DataFrame({})]
    Qr=[pd.DataFrame({})]
    Qr[0]=pd.read_csv(RECEIVE_SHEET_NAME,names=heading)
    Qs[0]=pd.read_csv(SEND_SHEET_NAME,names=heading)
    init.delete_diff(Qr,Qs)
    Qr[0]=init.init(Qr[0],'r')
    Qs[0]=init.init(Qs[0],'s')
    df_unique=init.append_unique(Qr,Qs)
    Qr_unique=df_unique['Qr']
    Qs_unique=df_unique['Qs']

    for way in range(2):
        if way==0:
            t_way=2
            combination=0 #0の場合は既存研究の方法、1の場合は組み合わせを考慮した手法
            ev_table=pre_results
        elif way==1:
            t_way=2
            combination=1
            ev_table=new_results
            Qr=Qr[:1]
            Qs=Qs[:1]
        else:
            print('プログラムを終了します') # shoudn't reach here
            break


        number=0 # 実験の回数
        check_digit=0

        race_set={}


        start = time.time()
        print("Creating race set",end='')
        race_set=race.creating_race_set(Qr,Qs,Qs_unique,race_set,combination)
        elapsed_time = time.time() - start
        print ("\nCreating race set took:{:.4g}".format(elapsed_time) + "[sec]")
        pprint.pprint((race_set))


        start = time.time()
        print("Creating race table",end='')
        table=tb.construct_race_table(Qs,Qr,race_set,t_way)
        if t_way>1:
            table=tb.expand_table(Qr,race_set,table,t_way)
        elapsed_time = time.time() - start
        print ("\nCreating race table took:{:.4g}".format(elapsed_time) + "[sec]")
        table=table.astype('int64')

        check_digit=1

        start = time.time()
        results=test.create_new_testcase(number,check_digit,Qr,Qs,table,race_set,Qr_unique,Qs_unique)
        Qr=results['recv']
        Qs=results['send']
        elapsed_time = time.time() - start
        print ("Creating test case took:{:.4g}".format(elapsed_time) + "[sec]")
        print('The number of Test Case is {}.'.format(len(table)))


        evaluation=ev.evaluate(Qr,Qs)
        Q=pd.concat([Qs[0],Qr[0]]).sort_index()
        
        if way==0:
            pre_results=pre_results.append(evaluation,ignore_index=True)
        elif way==1:
            new_results=new_results.append(evaluation,ignore_index=True)
        else:
            print('error')
        
_os_.save_file(RESULTS_FILE_NAME,results_path,pre_results,new_results)


In [14]:
table

""


In [13]:
Qs

[    ID Thread      Port Event  Index
 0   s1     T2  16955116  send      1
 1   s2     T2  16957212  send      2
 2   s3     T2  16955116  send      3
 3   s4     T2  16957212  send      4
 4   s5     T2  16955116  send      5
 5   s6     T2  16957212  send      6
 6   s7     T2  16955116  send      7
 7   s8     T2  16957212  send      8
 8   s9     T2  16955116  send      9
 9  s10     T2  16957212  send     10]

In [5]:
Qr[0].Thread

0    T2
1    T2
2    T3
3    T3
Name: Thread, dtype: object

In [3]:
loop=0
i=0
l=0
j=1
T='T1'
V='p2'
Q=pd.concat([Qs[loop],Qr[loop]]).sort_index()

In [4]:
array=Q[((Q.Thread==T[i]) & (Q.Port==V[l])) | ((Q.Thread==T[j]) &(Q.Port==V[l]))]

In [5]:
array

,ID,Thread,Port,Event,Index,KEY


In [472]:
Q=pd.concat([Qs[loop],Qr[loop]]).reset_index(drop=True)
Q

,ID,Thread,Port,Event,Index,coler
0,s2,T4,p2,send,1,NaN
1,s1,T1,p2,send,1,NaN
2,s4,T4,p3,send,2,NaN
3,s5,T2,p3,send,3,NaN
4,r1,T2,p2,res,1,white
5,r6,T2,p2,res,2,white
6,r3,T3,p3,res,1,white
7,r5,T3,p3,res,2,white


In [199]:
Q=pd.concat([Qs[0],Qr[0]]).sort_index()
V=Qr[0].Port.unique()
T=Q.Thread.unique()
V_total_number=len(V)
T_total_number=len(T)

total_number=0 # いったん0→後PNに書き換える

for l in range(V_total_number):
    for i in range(0,T_total_number-1):
        PNi=len(Q[(Q.Thread==T[i]) & (Q.Port==V[l])])-1
        if PNi==0:
            continue
        else:
            Nj=len(Q[(Q.Thread!=T[i]) & (Q.Port==V[l])])
            total_number+=PNi*Nj+PNi

value=0
for l in range(V_total_number):
    for i in range(0,T_total_number-1):
        for loop in range(len(Qr)):
            Q=pd.concat([Qs[loop],Qr[loop]]).sort_index()
            df=Q[(Q.Thread==T[i]) & (Q.Port==V[l])]
            if len(df)<=1:
                continue
            else:
                array=[]
                for j in range(len(df)-1):
                    array.append([])
                    temp=Q[df.iloc[j].name+1:df.iloc[j+1].name]
                    if len(temp)==0:
                        array[j].append(0)
                    for k in range(len(temp)):
                        array[j].append(temp.iloc[k].ID)
                    array[j]=list(set(array[j]))
                    value+=len(array[j])


In [73]:
race_set={}
combination_table=pd.DataFrame({},columns=['PORT','SEND','RECV'],dtype='object')
global Qr
global Qs
# race_set の初期化
for i in range(len(Qr[0])):

    race_set[Qr[0].iloc[i].ID]=[]

# sでループ
for i in range(len(Qs[0])):
    #sでループ

    number=Qs[0].iloc[i].name
    ID=Qs[0].iloc[i].ID
    thread=Qs[0].iloc[i].Thread
    port=Qs[0].iloc[i].Port

    pair_event=Qr[0].iloc[number]
    
    Qr_temp=Qr[0][(Qr[0].Thread==pair_event.Thread) & (Qr[0].Port==pair_event.Port) & (Qr[0].Index<pair_event.Index)] # race_setの候補
    for j in range(len(Qr_temp)):
        s_dash=Qs[0].loc[Qr_temp.iloc[j].name]
        if s_dash.ID not in Qs_unique[Qs_unique.ID==ID].iloc[0].cstruct:
            if combination==1:
                if combination_check(combination_table,port,thread,s_dash.Thread,pair_event.Thread,Qr_temp.iloc[j].Thread):
                    race_set[Qr_temp.iloc[j].ID].append(ID)
                    send_set='['+thread+','+s_dash.Thread+']'
                    recv_set='['+pair_event.Thread+','+Qr_temp.iloc[j].Thread+']'
                    data=[str(port),send_set,recv_set]
                    append_table=pd.DataFrame(data=[data],columns=['PORT','SEND','RECV'],dtype='object')
                    combination_table=combination_table.append(append_table,ignore_index=True)
            elif combination==0:
                race_set[Qr_temp.iloc[j].ID].append(ID)


    #これ以降はrの次がrだったとき

{'r1': ['s2'], 'r2': [], 'r3': ['s4'], 'r4': []}


In [108]:
combination_table.iloc[0]

0

In [55]:
Qs_unique

,ID,Thread,Port,Event,Index,cstruct
0,s1,T1,p2,send,1,
1,s2,T3,p2,send,1,
2,s3,T4,p2,send,1,
3,s4,T1,p2,send,2,[s1]
4,s5,T1,p2,send,3,"[s1, s4]"


In [9]:
import evaluation as ev

In [10]:
ev.hello()

hello


In [29]:
t_way

1

In [47]:
def cstruct(event,results):
    global Q
    global Qs
    global Qr
    global check_digit
    
    if check_digit==0:
        magic=0
    else:
        magic=1
    
    thread=event['Thread']
    index=event['Index']
       
    if index==1:
        return ''
    for i in range(index-1,0,-1):
        try:
            temp=Qr[number+magic][(Qr[number+magic].Thread == thread) & (Qr[number+magic].Index==i)].iloc[0]
        except:
            temp=Qs[number+magic][(Qs[number+magic].Thread == thread) & (Qs[number+magic].Index==i)].iloc[0]
        if temp['Event']=='send':
            results.append(temp['ID'])
            print('true:{}'.format(temp['ID']))
            event=temp
            results.extend(cstruct(event,results))
        else:
            results.append(temp['ID'])
            send=Qs[number+magic].iloc[temp.name]
            results.append(send['ID'])
            print('else:{}'.format(send['ID']))
            results.extend(cstruct(send,results))
    return list(set(results))


In [48]:
i=0
check_digit=0

In [12]:
Qr_unique

,ID,Thread,Port,Event,Index,coler,cstruct
0,r1,T2,[p2],res,1,white,
1,r2,T2,[p2],res,2,white,"[s1, r1]"
2,r3,T3,[p3],res,1,white,
3,r4,T3,[p3],res,2,white,"[r3, s3, s2, r2, s1, r1]"
4,r5,T3,[p3],res,2,white,"[r3, s4, s2]"
5,r6,T2,[p2],res,2,white,"[s2, r1]"
6,r7,T3,[p3],res,2,white,"[r3, r6, s3, s2, s1, r1]"


In [40]:
table

,r1,r3
0,0,1
1,1,0
2,1,1
